In [1]:
import xarray as xr
import numpy as np
import glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import netCDF4
import pandas as pd
# import xesmf as xe
from scipy import signal

In [2]:
data      = xr.load_dataset("/work/uo1075/u241321/data/thetao_ly_1969-2019_hindcast_80_step1.nc",decode_times=False) 
# data

In [3]:
ds = data ['thetao'][:,:,0:21,:,:]
# ds

In [4]:
# A reshaped DataArray. In the example in the function comment it would correspond to an array like

#In[1] Ar.dims
#Out[1]: ('year', 'month', 'lat', 'lon')

def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """


    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind
    
    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)




In [5]:
ds_r = xr_reshape(ds, 'ensemble', ['member', 'lead_year'], [np.arange(1,81),np.arange(1,11),])

In [6]:
annual_prediction = ds_r.transpose("time", "member", "lead_year", "depth", "lat", "lon") # 
dat = np.mean(annual_prediction,axis=0)
dif = annual_prediction-dat
# dif

In [7]:
dif.to_netcdf("/work/uo1075/u241321/data/thetao_ly_1970-2019_hindcast80.nc")

In [7]:
dropped = annual_prediction.stack(feature=("member","lead_year","depth","lat","lon")).dropna(dim="feature")
detrend = signal.detrend(dropped ,axis=0)
feature = dropped .coords["feature"]
time = dropped .coords["time"]
detrend = xr.DataArray(detrend, dims = ["time","feature"], coords = {"time":time,"feature":feature}).unstack()
detrend.to_netcdf("/work/uo1075/u241321/data/thetao_ly_1970-2019_hindcast80_dt.nc")